I am going to try and answer some of these questions given by the author of the dataset.


The data-set aims to answer the following key questions:

    1. Does various predicting factors which has been chosen initially really affect the Life expectancy? What are the predicting variables actually affecting the life expectancy?
    2. Should a country having a lower life expectancy value(<65) increase its healthcare expenditure in order to improve its average lifespan?
    3. How does Infant and Adult mortality rates affect life expectancy?
    4. Does Life Expectancy has positive or negative correlation with eating habits, lifestyle, exercise, smoking, drinking alcohol etc.
    5. What is the impact of schooling on the lifespan of humans?
    6. Does Life Expectancy have positive or negative relationship with drinking alcohol?
    7. Do densely populated countries tend to have lower life expectancy?
    8. What is the impact of Immunization coverage on life Expectancy?


In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder


data = pd.read_csv("C:/Users/Jibin Im/Downloads/Life Expectancy Data.csv")

data.head()


,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


In [53]:
data['Country'] = LabelEncoder().fit_transform(data['Country'])
data['Status'] = LabelEncoder().fit_transform(data['Status'])

#check for missing parts in the dataset
data.info()
#There are nulls in the dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2938 entries, 0 to 2937
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Country                          2938 non-null   int32  
 1   Year                             2938 non-null   int64  
 2   Status                           2938 non-null   int32  
 3   Life expectancy                  2928 non-null   float64
 4   Adult Mortality                  2928 non-null   float64
 5   infant deaths                    2938 non-null   int64  
 6   Alcohol                          2744 non-null   float64
 7   percentage expenditure           2938 non-null   float64
 8   Hepatitis B                      2385 non-null   float64
 9   Measles                          2938 non-null   int64  
 10   BMI                             2904 non-null   float64
 11  under-five deaths                2938 non-null   int64  
 12  Polio               

In [54]:
# for some reason some colujmn names have a space at the end?
data.columns

Index(['Country', 'Year', 'Status', 'Life expectancy ', 'Adult Mortality',
       'infant deaths', 'Alcohol', 'percentage expenditure', 'Hepatitis B',
       'Measles ', ' BMI ', 'under-five deaths ', 'Polio', 'Total expenditure',
       'Diphtheria ', ' HIV/AIDS', 'GDP', 'Population',
       ' thinness  1-19 years', ' thinness 5-9 years',
       'Income composition of resources', 'Schooling'],
      dtype='object')

In [55]:
#It is probably better to dorp na since getting median/mean from different countries can really mess up consistency 
#data = data.dropna()

#missing values with the mean within each country(didnt work)
data['Population'] = data.groupby('Country')['Population'].transform(lambda x: x.fillna(x.mean()))

#missing values with the mean within each country(worked)
data['Hepatitis B'] = data.groupby('Country')['Hepatitis B'].transform(lambda x: x.fillna(x.mean()))

#just drop rest of na
data = data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1974 entries, 0 to 2937
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Country                          1974 non-null   int32  
 1   Year                             1974 non-null   int64  
 2   Status                           1974 non-null   int32  
 3   Life expectancy                  1974 non-null   float64
 4   Adult Mortality                  1974 non-null   float64
 5   infant deaths                    1974 non-null   int64  
 6   Alcohol                          1974 non-null   float64
 7   percentage expenditure           1974 non-null   float64
 8   Hepatitis B                      1974 non-null   float64
 9   Measles                          1974 non-null   int64  
 10   BMI                             1974 non-null   float64
 11  under-five deaths                1974 non-null   int64  
 12  Polio                    

In [56]:
X = data.drop(columns='Life expectancy ')
y = data['Life expectancy ']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
data.shape[0]

1974

In [57]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
clf1 = RandomForestRegressor()
clf2 = GradientBoostingRegressor()

clf3 = VotingRegressor(estimators=[('RFR', clf1), ('GBR', clf2)],
                        weights=[0.5, 0.5])


In [58]:
clf3.fit(X_train, y_train)

pred = clf3.predict(X_test)
r2_score = r2_score(y_test, pred)
mean_squared_error = mean_squared_error(y_test, pred, squared=False)
print(f'r2_score: {r2_score:.4f}, mean_squared_error:{mean_squared_error:.4f}')

r2_score: 0.9425, mean_squared_error:2.3398


c:\Users\Jibin Im\anaconda3\envs\pytorch\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
